# Performance investigation: Pyodide

References: 
* [https://gist.github.com/rth/c71fe792eb56fb271317e35e08576c7a#results]()
* [https://github.com/pyodide/pyodide/tree/main/benchmark/benchmarks]()

![](https://pbs.twimg.com/media/FDum1WfUUAADPMx?format=png&name=medium)

[_Norman-Nielsen Group_](https://www.nngroup.com/articles/response-times-3-important-limits/)

In [ ]:
import numpy as np
from time import time

N = 1000
X = np.random.RandomState(0).rand(N, N)
t0 = time()
X.dot(X)
print(f'Wall time: {time() - t0:.2f} s')

Wall time: 2.36 s


In [ ]:
import numpy as np


def allpairs_distances(A, B):
    """This returns the euclidean distances squared
    dist2(x, y) = dot(x, x) - 2 * dot(x, y) + dot(y, y)
    """
    A2 = np.einsum("ij,ij->i", A, A)
    B2 = np.einsum("ij,ij->i", B, B)
    return A2[:, None] + B2[None, :] - 2 * np.dot(A, B.T)

In [ ]:
import numpy as np


def allpairs_distances_loops(X, Y):
    result = np.zeros((X.shape[0], Y.shape[0]), X.dtype)
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            result[i, j] = np.sum((X[i, :] - Y[j, :]) ** 2)
    return result

In [ ]:
import numpy as np


def arc_distance(theta_1, phi_1, theta_2, phi_2):
    """
    Calculates the pairwise arc distance between all points in vector a and b.
    """
    temp = (
        np.sin((theta_2 - theta_1) / 2) ** 2
        + np.cos(theta_1) * np.cos(theta_2) * np.sin((phi_2 - phi_1) / 2) ** 2
    )
    distance_matrix = 2 * (np.arctan2(np.sqrt(temp), np.sqrt(1 - temp)))
    return distance_matrix

In [ ]:
import numpy as np


def check_mask(db, mask=[1, 0, 1]):
    out = np.zeros(db.shape[0], dtype=bool)
    for idx, line in enumerate(db):
        target, vector = line[0], line[1:]
        if (mask == np.bitwise_and(mask, vector)).all():
            if target == 1:
                out[idx] = 1
    return out

In [ ]:
import numpy as np


def create_grid(x):
    N = x.shape[0]
    z = np.zeros((N, N, 3))
    z[:, :, 0] = x.reshape(-1, 1)
    z[:, :, 1] = x
    fast_grid = z.reshape(N * N, 3)
    return fast_grid

In [ ]:
def cronbach(itemscores):
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)
    return nitems / (nitems - 1) * (1 - itemvars.sum() / tscores.var(ddof=1))

In [ ]:
def diffusion(u, tempU, iterNum):
    """
    Apply Numpy matrix for the Forward-Euler Approximation
    """
    mu = 0.1

    for n in range(iterNum):
        tempU[1:-1, 1:-1] = u[1:-1, 1:-1] + mu * (
            u[2:, 1:-1]
            - 2 * u[1:-1, 1:-1]
            + u[0:-2, 1:-1]
            + u[1:-1, 2:]
            - 2 * u[1:-1, 1:-1]
            + u[1:-1, 0:-2]
        )
        u[:, :] = tempU[:, :]
        tempU[:, :] = 0.0

In [ ]:
import numpy as np


def laplacian(grid):
    return (
        np.roll(grid, +1, 0)
        + np.roll(grid, -1, 0)
        + np.roll(grid, +1, 1)
        + np.roll(grid, -1, 1)
        - 4 * grid
    )


def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)

In [ ]:
import numpy as np


def fdtd(input_grid, steps):
    grid = input_grid.copy()
    old_grid = np.zeros_like(input_grid)
    previous_grid = np.zeros_like(input_grid)

    l_x = grid.shape[0]
    l_y = grid.shape[1]

    for i in range(steps):
        np.copyto(previous_grid, old_grid)
        np.copyto(old_grid, grid)

        for x in range(l_x):
            for y in range(l_y):
                grid[x, y] = 0.0
                if 0 < x + 1 < l_x:
                    grid[x, y] += old_grid[x + 1, y]
                if 0 < x - 1 < l_x:
                    grid[x, y] += old_grid[x - 1, y]
                if 0 < y + 1 < l_y:
                    grid[x, y] += old_grid[x, y + 1]
                if 0 < y - 1 < l_y:
                    grid[x, y] += old_grid[x, y - 1]

                grid[x, y] /= 2.0
                grid[x, y] -= previous_grid[x, y]

    return grid

In [ ]:
import numpy as np


def fft(x):
    return np.fft(x)

In [ ]:
import numpy as np


def grayscott(counts, Du, Dv, F, k):
    n = 100
    U = np.zeros((n + 2, n + 2), dtype=np.float32)
    V = np.zeros((n + 2, n + 2), dtype=np.float32)
    u, v = U[1:-1, 1:-1], V[1:-1, 1:-1]

    r = 20
    u[:] = 1.0
    U[n // 2 - r : n // 2 + r, n // 2 - r : n // 2 + r] = 0.50
    V[n // 2 - r : n // 2 + r, n // 2 - r : n // 2 + r] = 0.25
    u += 0.15 * np.random.random((n, n))
    v += 0.15 * np.random.random((n, n))

    for i in range(counts):
        Lu = (
            U[0:-2, 1:-1]
            + U[1:-1, 0:-2]
            - 4 * U[1:-1, 1:-1]
            + U[1:-1, 2:]
            + U[2:, 1:-1]
        )
        Lv = (
            V[0:-2, 1:-1]
            + V[1:-1, 0:-2]
            - 4 * V[1:-1, 1:-1]
            + V[1:-1, 2:]
            + V[2:, 1:-1]
        )
        uvv = u * v * v
        u += Du * Lu - uvv + F * (1 - u)
        v += Dv * Lv + uvv - (F + k) * v

    return V

In [ ]:
def grouping(values):
    import numpy as np

    diff = np.concatenate(([1], np.diff(values)))
    idx = np.concatenate((np.where(diff)[0], [len(values)]))
    return values[idx[:-1]], np.diff(idx)

In [ ]:
import math


def window_floor(idx, radius):
    if radius > idx:
        return 0
    else:
        return idx - radius


def window_ceil(idx, ceil, radius):
    if idx + radius > ceil:
        return ceil
    else:
        return idx + radius


def growcut(image, state, state_next, window_radius):
    changes = 0
    sqrt_3 = math.sqrt(3.0)

    height = image.shape[0]
    width = image.shape[1]

    for j in range(width):
        for i in range(height):

            winning_colony = state[i, j, 0]
            defense_strength = state[i, j, 1]

            for jj in range(
                window_floor(j, window_radius), window_ceil(j + 1, width, window_radius)
            ):
                for ii in range(
                    window_floor(i, window_radius),
                    window_ceil(i + 1, height, window_radius),
                ):
                    if ii != i and jj != j:
                        d = image[i, j, 0] - image[ii, jj, 0]
                        s = d * d
                        for k in range(1, 3):
                            d = image[i, j, k] - image[ii, jj, k]
                            s += d * d
                        gval = 1.0 - math.sqrt(s) / sqrt_3

                        attack_strength = gval * state[ii, jj, 1]

                        if attack_strength > defense_strength:
                            defense_strength = attack_strength
                            winning_colony = state[ii, jj, 0]
                            changes += 1

            state_next[i, j, 0] = winning_colony
            state_next[i, j, 1] = defense_strength

    return changes

In [ ]:
def harris(X):
    m, n = X.shape
    dx = (X[1:, :] - X[: m - 1, :])[:, 1:]
    dy = (X[:, 1:] - X[:, : n - 1])[1:, :]

    #
    #   At each point we build a matrix
    #   of derivative products
    #   M =
    #   | A = dx^2     C = dx * dy |
    #   | C = dy * dx  B = dy * dy |
    #
    #   and the score at that point is:
    #      det(M) - k*trace(M)^2
    #
    A = dx * dx
    B = dy * dy
    C = dx * dy
    tr = A + B
    det = A * B - C * C
    k = 0.05
    return det - k * tr * tr

In [ ]:
import numpy as np


def hasting(y, t, a1, a2, b1, b2, d1, d2):
    yprime = np.empty((3,))
    yprime[0] = y[0] * (1.0 - y[0]) - a1 * y[0] * y[1] / (1.0 + b1 * y[0])
    yprime[1] = (
        a1 * y[0] * y[1] / (1.0 + b1 * y[0])
        - a2 * y[1] * y[2] / (1.0 + b2 * y[1])
        - d1 * y[1]
    )
    yprime[2] = a2 * y[1] * y[2] / (1.0 + b2 * y[1]) - d2 * y[2]
    return yprime

In [ ]:
import numpy as np


def hyantes(xmin, ymin, xmax, ymax, step, range_, range_x, range_y, t):
    X, Y = t.shape
    pt = np.zeros((X, Y))
    for i in range(X):
        for j in range(Y):
            for k in t:
                tmp = 6368.0 * np.arccos(
                    np.cos(xmin + step * i)
                    * np.cos(k[0])
                    * np.cos((ymin + step * j) - k[1])
                    + np.sin(xmin + step * i) * np.sin(k[0])
                )
                if tmp < range_:
                    pt[i, j] += k[2] / (1 + tmp)
    return pt

In [ ]:
import numpy as np


def kernel(zr, zi, cr, ci, lim, cutoff):
    """Computes the number of iterations `n` such that
    |z_n| > `lim`, where `z_n = z_{n-1}**2 + c`.
    """
    count = 0
    while ((zr * zr + zi * zi) < (lim * lim)) and count < cutoff:
        zr, zi = zr * zr - zi * zi + cr, 2 * zr * zi + ci
        count += 1
    return count


def julia(cr, ci, N, bound=1.5, lim=1000.0, cutoff=1e6):
    """Pure Python calculation of the Julia set for a given `c`.  No NumPy
    array operations are used.
    """
    julia = np.empty((N, N), np.uint32)
    grid_x = np.linspace(-bound, bound, N)
    for i, x in enumerate(grid_x):
        for j, y in enumerate(grid_x):
            julia[i, j] = kernel(x, y, cr, ci, lim, cutoff)
    return julia

In [ ]:
import numpy as np


def l2norm(x):
    return np.sqrt(np.einsum("ij,ij->i", x, x))

In [ ]:
def large_decimal_list(A, B):
    return [a * b for a, b in zip(A, B)]

In [ ]:
import numpy as np


def wrap(pos, offset, bound):
    return (pos + offset) % bound


def clamp(pos, offset, bound):
    return min(bound - 1, max(0, pos + offset))


def reflect(pos, offset, bound):
    idx = pos + offset
    return min(2 * (bound - 1) - idx, max(idx, -idx))


def local_maxima(data, mode=wrap):
    wsize = data.shape
    result = np.ones(data.shape, bool)
    for pos in np.ndindex(data.shape):
        myval = data[pos]
        for offset in np.ndindex(wsize):
            neighbor_idx = tuple(
                mode(p, o - w // 2, w) for (p, o, w) in zip(pos, offset, wsize)
            )
            result[pos] &= data[neighbor_idx] <= myval
    return result

In [ ]:
import numpy

# pythran export log_likelihood(float64[], float64, float64)


def log_likelihood(data, mean, sigma):
    s = (data - mean) ** 2 / (2 * (sigma ** 2))
    pdfs = numpy.exp(-s)
    pdfs /= numpy.sqrt(2 * numpy.pi) * sigma
    return numpy.log(pdfs).sum()

In [ ]:
import numpy as np


def lstsqr(x, y):
    """Computes the least-squares solution to a linear matrix equation."""
    x_avg = np.average(x)
    y_avg = np.average(y)
    dx = x - x_avg
    var_x = np.sum(dx ** 2)
    cov_xy = np.sum(dx * (y - y_avg))
    slope = cov_xy / var_x
    y_interc = y_avg - slope * x_avg
    return (slope, y_interc)

In [ ]:
def kernel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return max_iters


def mandel(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = kernel(real, imag, iters)
            image[y, x] = color

In [ ]:
import numpy as np


def multiple_sum(array):

    rows = array.shape[0]
    cols = array.shape[1]

    out = np.zeros((rows, cols))

    for row in range(0, rows):
        out[row, :] = np.sum(array - array[row, :], 0)

    return out

In [ ]:
import numpy as np


def pairwise_loop(X):
    M, N = X.shape
    D = np.empty((M, M))
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [ ]:
import numpy as np


def periodic_dist(x, y, z, L, periodicX, periodicY, periodicZ):
    """Computes distances between all particles and places the result
    in a matrix such that the ij th matrix entry corresponds to the
    distance between particle i and j"""
    N = len(x)
    xtemp = np.tile(x, (N, 1))
    dx = xtemp - xtemp.T
    ytemp = np.tile(y, (N, 1))
    dy = ytemp - ytemp.T
    ztemp = np.tile(z, (N, 1))
    dz = ztemp - ztemp.T

    # Particles 'feel' each other across the periodic boundaries
    if periodicX:
        dx[dx > L / 2] = dx[dx > L / 2] - L
        dx[dx < -L / 2] = dx[dx < -L / 2] + L

    if periodicY:
        dy[dy > L / 2] = dy[dy > L / 2] - L
        dy[dy < -L / 2] = dy[dy < -L / 2] + L

    if periodicZ:
        dz[dz > L / 2] = dz[dz > L / 2] - L
        dz[dz < -L / 2] = dz[dz < -L / 2] + L

    # Total Distances
    d = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

    # Mark zero entries with negative 1 to avoid divergences
    d[d == 0] = -1

    return d, dx, dy, dz

In [ ]:
import numpy as np


def repeating(x, nvar_y):
    nvar_x = x.shape[0]
    y = np.empty(nvar_x * (1 + nvar_y))
    y[0:nvar_x] = x[0:nvar_x]
    y[nvar_x:] = np.repeat(x, nvar_y)
    return y

In [ ]:
import numpy as np


def reverse_cumsum(x):
    return np.cumsum(x[::-1])[::-1]

In [ ]:
import numpy as np

# pythran export rosen(float[])


def rosen(x):
    t0 = 100 * (x[1:] - x[:-1] ** 2) ** 2
    t1 = (1 - x[:-1]) ** 2
    return np.sum(t0 + t1)

In [ ]:
from numpy import zeros, power, tanh


def slowparts(d, re, preDz, preWz, SRW, RSW, yxV, xyU, resid):
    """computes the linear algebra intensive part of the gradients of the grae"""

    def fprime(x):
        return 1 - power(tanh(x), 2)

    partialDU = zeros((d + 1, re, 2 * d, d))
    for k in range(2 * d):
        for i in range(d):
            partialDU[:, :, k, i] = (
                fprime(preDz[k])
                * fprime(preWz[i])
                * (SRW[i, k] + RSW[i, k])
                * yxV[:, :, i]
            )

    return partialDU

In [ ]:
def smoothing(x, alpha):
    """
    Exponential smoothing of a time series
    For x = 10**6 floats
    - Python runtime: 9 seconds
    - Parakeet runtime: .01 seconds
    """
    s = x.copy()
    for i in range(1, len(x)):
        s[i] = alpha * x[i] + (1 - alpha) * s[i - 1]
    return s

In [ ]:
def specialconvolve(a):
    # sorry, you must pad the input yourself
    rowconvol = a[1:-1, :] + a[:-2, :] + a[2:, :]
    colconvol = (
        rowconvol[:, 1:-1] + rowconvol[:, :-2] + rowconvol[:, 2:] - 9 * a[1:-1, 1:-1]
    )
    return colconvol

In [ ]:
import numpy


def vibr_energy(harmonic, anharmonic, i):
    return numpy.exp(-harmonic * i - anharmonic * (i ** 2))

In [ ]:
import numpy as np


def physics(masspoints, dt, plunk, which):
    ppos = masspoints[1]
    cpos = masspoints[0]
    N = cpos.shape[0]
    # apply hooke's law
    HOOKE_K = 2100000.0
    DAMPING = 0.0001
    MASS = 0.01

    force = np.zeros((N, 2))
    for i in range(1, N):
        dx, dy = cpos[i] - cpos[i - 1]
        dist = np.sqrt(dx ** 2 + dy ** 2)
        assert dist != 0
        fmag = -HOOKE_K * dist
        cosine = dx / dist
        sine = dy / dist
        fvec = np.array([fmag * cosine, fmag * sine])
        force[i - 1] -= fvec
        force[i] += fvec

    force[0] = force[-1] = 0, 0
    force[which][1] += plunk
    accel = force / MASS

    # verlet integration
    npos = (2 - DAMPING) * cpos - (1 - DAMPING) * ppos + accel * (dt ** 2)

    masspoints[1] = cpos
    masspoints[0] = npos


# pythran export wave(int)


def wave(PARTICLE_COUNT):
    SUBDIVISION = 300
    FRAMERATE = 60
    count = PARTICLE_COUNT
    width, height = 1200, 400

    masspoints = np.empty((2, count, 2), np.float64)
    initpos = np.zeros(count, np.float64)
    for i in range(1, count):
        initpos[i] = initpos[i - 1] + float(width) / count
    masspoints[:, :, 0] = initpos
    masspoints[:, :, 1] = height / 2
    f = 15
    plunk_pos = count // 2
    physics(masspoints, 1.0 / (SUBDIVISION * FRAMERATE), f, plunk_pos)
    return masspoints[0, count // 2]